In [ ]:
# rye add --git https://github.com/ctgk/PRML.git prml

import seaborn as sns
import math

# Apply the default theme
sns.set_theme()


In [ ]:
import prml
import numpy as np
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt

%matplotlib inline

from prml.preprocess import GaussianFeature, PolynomialFeature, SigmoidalFeature
from prml.linear import (
    BayesianRegression,
    EmpiricalBayesRegression,
    LinearRegression,
    RidgeRegression,
)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split


def create_toy_data(func, sample_size, std, domain=[0, 1]):
    rng = np.random.default_rng()
    x = np.linspace(domain[0], domain[1], sample_size)
    # x = rng.uniform(0, 1, sample_size)
    np.random.shuffle(x)

    y = func(x) + rng.normal(scale=std, size=x.shape)
    return x, y


def sinusoidal(x):
    return np.sin(2 * np.pi * x)


m = 20
x, y = create_toy_data(sinusoidal, m, 0.25)

# Reshape x to work with sklearn (needed if x is a 1D array)
x = x.reshape(-1, 1)

# Split dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Print the shapes of the splits
print("Training set size:", x_train.shape, y_train.shape)
print("Test set size:", x_test.shape, y_test.shape)


In [ ]:
plt.figure(figsize=[10, 8])
plt.scatter(x_train, y_train, facecolor="none", edgecolor="b", s=50, label="training data")
plt.scatter(x_test, y_test, facecolor="none", edgecolor="r", label="test data")
plt.legend()
plt.show()

In [ ]:
M = 9

feature = PolynomialFeature(M)

X_train = feature.transform(x_train)
X_test = feature.transform(x_test)
model = LinearRegression()
model.fit(X_train, y_train)

plt.figure(figsize=[10, 8])
plt.plot(model.w)
plt.xlabel("index of $w$")
plt.ylabel("$w$")


plt.figure(figsize=[10, 8])

# training data
plt.scatter(x_train, y_train, facecolor="none", edgecolor="b", s=50, label="training data")
# test data
plt.scatter(x_test, y_test, facecolor="none", edgecolor="r", s=50, label="test data")

# M=9 polynomial regression hypothesis
x_range = np.linspace(X_train.min(), X_train.max(), 100).reshape(-1, 1)
X_range = feature.transform(x_range)
y_hat_range, y_hat_range_std = model.predict(X_range, return_std=True)

plt.plot(x_range, y_hat_range, label="Least Squares Prediction")

plt.legend()
plt.xlabel("$x$")
plt.ylabel("$y$ or $\hat{y}$")

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import test

import numpy as np
import matplotlib.pyplot as plt


# SGD Loop with polynomial features and mini-batch support
def sgd_loop(
    X_train,
    y_train,
    X_test,
    y_test,
    w,
    learning_rate,
    epochs,
    lambda_reg,
    n_samples,
    batch_size=1,
):
    # Arrays to store loss values for visualization
    losses = []
    test_losses = []

    # SGD Loop
    for epoch in range(epochs):
        for i in range(0, n_samples, batch_size):  # Iterate in mini-batches
            # Select a mini-batch of `batch_size`
            batch_indices = np.random.choice(n_samples, batch_size, replace=False)
            xi = X_train[batch_indices]
            yi = y_train[batch_indices]

            # Prediction
            y_pred = np.dot(xi, w)

            # Compute error
            error = y_pred - yi

            # Compute gradients (Mean of batch gradients)
            dw = (2 / batch_size) * (xi.T @ error).flatten() + 2 * lambda_reg * w

            # Update weights
            w -= learning_rate * dw

        # Compute training loss
        y_hat_train = X_train @ w
        loss = np.mean((y_hat_train - y_train) ** 2) + lambda_reg * np.sum(w**2)
        losses.append(loss)

        # Compute test loss
        y_hat_test = X_test @ w
        test_loss = np.mean((y_hat_test - y_test) ** 2) + lambda_reg * np.sum(w**2)
        test_losses.append(test_loss)

    # Plot loss vs epochs
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(epochs), losses, label="Training Loss", color="blue")
    plt.plot(range(epochs), test_losses, label="Test Loss", color="red")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training & Test Loss vs Epochs")
    plt.legend()

    # Start of plotting regression curve
    plt.subplot(1, 2, 2)
    x_range = np.linspace(X_train.min(), X_train.max(), 100).reshape(-1, 1)

    # computation
    hypothesis = np.zeros_like(x_range)
    for i in range(len(w)):
        hypothesis += w[i] * (x_range**i)  # Add each term: w_i * x^i

    # Plot training and testing data points
    plt.scatter(x_train, y_train, color="blue", label="Training Data")
    plt.scatter(x_test, y_test, color="red", label="Test Data")

    # plot our regression curve
    plt.plot(x_range, hypothesis, color="green", label="Regularized Hypothesis")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title("Regularized 9-degree Polynomial Regression Function")
    plt.legend()

    plt.tight_layout()
    plt.show()

    min_loss = min(losses)
    min_index = losses.index(min_loss)
    min_test_loss = min(test_losses)
    min_test_index = test_losses.index(min_test_loss)

    # Print final parameters
    print("Final weights:", w.flatten())
    print(f"Smallest Loss: {min_loss} loss at index {min_index}")
    print(f"Smallest Test Loss: {min_test_loss} loss at index {min_test_index}")

    return min(test_losses)


In [ ]:
n_features = X_train.shape[1]  # Number of polynomial features

# Replace this vector with the optimal trial vector
w = np.array(
    [
        0.3821733,
        0.3096256,
        -1.87535451,
        -0.41147161,
        0.48204058,
        -0.47616227,
        -0.91596271,
        0.33230879,
        0.92623104,
        1.34749729,
    ]
)

test_loss = sgd_loop(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    w=w,
    learning_rate=0.01,
    epochs=10000,
    lambda_reg=0.02,
    n_samples=len(y_train),
    batch_size=5,
)


In [ ]:
import optuna


def objective(trial):
    # Sample lambda_reg from Optuna (log-uniform for better scaling)
    lambda_reg = trial.suggest_loguniform("lambda_reg", 1e-4, 1.0)  # Define the search space

    # batch_size = trial.suggest_uniform(
    #     "batch_size", 1, 10
    # )  # Define the search space

    # Initialize weights
    w = np.random.randn(10)

    # fix batch size to the size of the training set
    batch_size = len(y_train)

    # Run SGD
    min_test_loss = sgd_loop(
        X_train=X_train,
        y_train=y_train,
        X_test=X_test,
        y_test=y_test,
        w=w,
        learning_rate=0.01,
        epochs=10000,
        lambda_reg=lambda_reg,
        n_samples=len(y_train),
        batch_size=batch_size,
    )

    return min_test_loss


In [ ]:
# Set direction to "minimize" as we want to minimize the objective
study = optuna.create_study(direction="minimize")

n_trials = 50

study.optimize(objective, n_trials=50)  # Run the optimization for 100 trials

# Best lambda_reg
best_lambda_reg = study.best_params["lambda_reg"]
print(f"Best lambda_reg: {best_lambda_reg}")
print(f"Best test loss: {study.best_value}")

# Best Batch size
best_lambda_reg = study.best_params["lambda_reg"]
print(f"Best lambda_reg: {best_lambda_reg}")
print(f"Best test loss: {study.best_value}")

# Plot optimization history
optuna.visualization.matplotlib.plot_optimization_history(study)
plt.show()
